# WSI - ćwiczenie 1.
### Zagadnienie przeszukiwania i podstawowe podejścia do niego


In [10]:
import numpy as np
import plotly.graph_objects as go

1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [11]:
def f(x):
    return 10*(x**4) + 3*(x**3) - 30*(x**2) + 10*x 
    
def g(vect):
    assert len(vect) == 2
    return 10*(vect[1]**4) + 10*(vect[0]**4) + 3*(vect[0]**3) - 30*(vect[0]**2) + 10*vect[0]

### obliczanie gradientu i hesjanu

### algorytm gradientowy

In [18]:
from tracemalloc import start


def gradient_descent(func, start_point, step_coef, end_treshold): # point is 2 dimensional
    steps = [start_point]
    # check if algorithm isnt oscilating
    # check if close enough to minimum
    point = start_point
    end = False
    i = 0
    while not end:
        np.subtract(point, np.gradient(point + [func(point)]))         
        steps.append(point)
        i+=1
        if i >= 100: 
            end = True

    return np.array(steps)

SyntaxError: invalid syntax (1626529091.py, line 4)

### generowanie wykresów

In [19]:
# This piece of code creates a 3d plot with easily controllable scale 
# (since the builtin feature is chaotic and hard to control for me), 
# and animation of algorythm's steps (also controlled by a slider)

class PlotGenerator:
    def __init__(self, 
                 alg_steps, 
                 min_scale_range = 1,
                 max_scale_range = 6,
                 scale_slider_step = 1, 
                 mesh_density = 100,
                 func = g)  -> None:
        self.min_scale_range = min_scale_range
        self.max_scale_range = max_scale_range
        self.scale_slider_step = scale_slider_step
        self.mesh_density = mesh_density
        self.func = func
        self.alg_steps = alg_steps

        self.x_steps = self.alg_steps[:,0]
        self.y_steps = self.alg_steps[:,1]
        self.z_steps = [func([x, y]) for x, y in zip(self.x_steps, self.y_steps)]

    def generate(self):
        fig = go.Figure()
        # add trace for each scale step
        for step in np.arange(self.min_scale_range, self.max_scale_range, self.scale_slider_step):
            # create plot's main surface (plot of g(x, y))
            x_space, y_space = 2*[np.linspace(-step, step, self.mesh_density)]
            f_surface_params = dict(z=[[self.func([x, y]) for y in y_space] for x in x_space], 
                                    x=x_space, 
                                    y=y_space, 
                                    visible=False,
                                    colorscale ='ice')
            # create step traces
            step_traces_params = dict(z=self.z_steps, 
                                    x=self.x_steps, 
                                    y=self.y_steps, 
                                    visible=False,
                                    name='alg steps',
                                    marker=dict(size=5,
                                                color=self.z_steps,
                                                colorscale='burg'))
            # add function surface plot
            fig.add_trace(go.Surface(f_surface_params))
            # add plot of algorithm's steps over the function surface
            fig.add_trace(go.Scatter3d(step_traces_params))

        # make first trace visible
        fig.data[0].visible, fig.data[1].visible = True, True


        # create scale slider
        scale_slider_steps = []
        for i in range(0, (len(fig.data)//2)-1):
            step = dict(method="update",
                        args=[{"visible": [False] * len(fig.data)}],
                        label='')
            step["args"][0]["visible"][2*i] = True
            step["args"][0]["visible"][2*i+1] = True
            scale_slider_steps.append(step)

        # create algorithm step slider
        step_slider_steps = []
        for i in range(len(self.z_steps)):
            step = dict(method="update",
                        args=[{"visible": [False] * len(fig.data)}],
                        label='')
            step["args"][0]["visible"][i] = True
            step_slider_steps.append(step)

        # add sliders
        sliders = [dict(steps=scale_slider_steps, yanchor="bottom", currentvalue=dict(visible=False))]
        # dict(steps=step_slider_steps, yanchor="middle", currentvalue=dict(visible=False)) 

        fig.update_layout(sliders=sliders, title='Plot of g(x, y)', autosize=False, height=600, width=600)
        fig.show()


plotter = PlotGenerator(gradient_descent(g, [2, 2], 1, 0.1))
plotter.generate()

ValueError: operands could not be broadcast together with shapes (2,) (3,) 